##EXTRACCIÓN DE CARACTERÍSTICAS

Detección de formas

In [ ]:
import cv2
import numpy as np
from skimage import io
from google.colab.patches import cv2_imshow

url = "https://i.imgur.com/s3zNM51.png"
image = io.imread(url)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
canny = cv2.Canny(gray, 10, 150)
canny = cv2.dilate(canny, None, iterations=1)
canny = cv2.erode(canny, None, iterations=1)
cnts,_ = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)# OpenCV 4

for c in cnts:
  epsilon = 0.01*cv2.arcLength(c,True)
  approx = cv2.approxPolyDP(c,epsilon,True)
  x,y,w,h = cv2.boundingRect(approx)
  if len(approx)==3:
    cv2.putText(image,'Triangulo', (x,y-5),1,1.5,(0,255,0),2)
  if len(approx)==4:
    aspect_ratio = float(w)/h
    print('aspect_ratio= ', aspect_ratio)
    if aspect_ratio == 1:
      cv2.putText(image,'Cuadrado', (x,y-5),1,1.5,(0,255,0),2)
    else:
      cv2.putText(image,'Rectangulo', (x,y-5),1,1.5,(0,255,0),2)
  if len(approx)==5:
    cv2.putText(image,'Pentagono', (x,y-5),1,1.5,(0,255,0),2)
  if len(approx)==6:
    cv2.putText(image,'Hexagono', (x,y-5),1,1.5,(0,255,0),2)
  if len(approx)>10:
    cv2.putText(image,'Circulo', (x,y-5),1,1.5,(0,255,0),2)
  cv2.drawContours(image, [approx], 0, (0,255,0),2)

cv2_imshow(image)

Detección de puntos de interés: SIFT

* Instalación de la última versión de OpenCV-Contrib

In [ ]:
!pip install opencv-contrib-python==4.4.0.44

In [ ]:
from random import randrange
import matplotlib.pyplot as plt
import numpy as np
import cv2

url2 = "https://raw.githubusercontent.com/YoniChechik/AI_is_Math/master/c_08_features/left.jpg"
image2 = io.imread(url2)
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
gray_l = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)
url3 = "https://raw.githubusercontent.com/YoniChechik/AI_is_Math/master/c_08_features/right.jpg"
image3 = io.imread(url3)
image3 = cv2.cvtColor(image3, cv2.COLOR_BGR2RGB)
gray_r = cv2.cvtColor(image3, cv2.COLOR_RGB2GRAY)

* Detección de características con SIFT y descripción

In [ ]:
# use orb if sift is not installed
feature_extractor = cv2.SIFT_create()

# find the keypoints and descriptors with chosen feature_extractor
kp_l, desc_l = feature_extractor.detectAndCompute(gray_l, None)
kp_r, desc_r = feature_extractor.detectAndCompute(gray_r, None)

test = cv2.drawKeypoints(image2, kp_l, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2_imshow(test)

* Emparejamiento de características únicas

In [ ]:
bf = cv2.BFMatcher()
matches = bf.knnMatch(desc_l, desc_r, k=2)

# Apply ratio test
good_match = []
for m in matches:
    if m[0].distance/m[1].distance < 0.5:
        good_match.append(m)
good_match_arr = np.asarray(good_match)

# show only 30 matches
im_matches = cv2.drawMatchesKnn(image2, kp_l, image3, kp_r,
                                good_match[0:100], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv2_imshow(im_matches)

* Encontrando la homografía entre las imagenes

In [ ]:
good_kp_l = np.array([kp_l[m.queryIdx].pt for m in good_match_arr[:, 0]]).reshape(-1, 1, 2)
good_kp_r = np.array([kp_r[m.trainIdx].pt for m in good_match_arr[:, 0]]).reshape(-1, 1, 2)
H, masked = cv2.findHomography(good_kp_r, good_kp_l, cv2.RANSAC, 5.0)

print(H)

[[ 7.64973552e-01  3.48655815e-02  4.46971205e+02]
 [-1.35238875e-01  9.09695982e-01  7.64486507e+01]
 [-2.10645007e-04 -3.52392896e-05  1.00000000e+00]]


* Superponiendo las imágenes

In [ ]:
rgb_r_warped = cv2.warpPerspective(image3, H, (image2.shape[1] + image3.shape[1], image2.shape[0]))
rgb_r_warped[0:image2.shape[0], 0:image2.shape[1]] = image2

cv2_imshow(rgb_r_warped)

##TÓPICOS AVANZADOS

Detección de rostros

In [ ]:
import cv2
import numpy as np
from skimage import io
from google.colab.patches import cv2_imshow

face_cascade = cv2.CascadeClassifier("./sample_data/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("./sample_data/haarcascade_eye.xml")

url = "https://canadianfriendsofperu.files.wordpress.com/2012/08/cropped-dsc018831.jpg"
#url = "https://voluntariadodandomas.files.wordpress.com/2020/10/43385.jpg"
image = io.imread(url)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

cv2_imshow(image)

faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
  cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
  roi_gray = gray[y:y+h, x:x+w]
  roi_color = image[y:y+h, x:x+w]
  eyes = eye_cascade.detectMultiScale(roi_gray)
  for (ex,ey,ew,eh) in eyes:
    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

print("")

cv2_imshow(image)